# Train data

In [ ]:
# 카테고리 변수들 
# 변수 선택 : 카테코리 수가 많이 없고, 분포가 한곳에 치우져지지 않고 고루 분포되어 있는 변수들 선택
# 결측값 : None 으로 통일 (실제 data 에서도 None으로 지정이 되어있음)
add_x = pd.concat([x_train,x_test])
one_hot = pd.get_dummies(add_x)
train_onehot=one_hot.iloc[0:1460,:]
test_onehot=one_hot.iloc[1460:2920,:]
# 수치형 변수들
# 변수 선택 : 종속변수인 SalePrice 와 상관계수가 0.45 이상인 변수들 선택
# 결측값 : 평균으로 처리
x_data_train = pd.concat([train_onehot,x_data], axis=1)
# MinMaxScaler
# train 데이터를 기준으로  설정
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data =scaler.transform(x_data_train)
y_data = df_train.iloc[:,-1]
y_data = y_data.values
y_data=y_data.reshape(-1,1)

X = tf.placeholder(tf.float32, shape=[None,70])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([70,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(200001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 2000 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

# Test data

In [ ]:
# 수치형 변수 결측값 : 평균으로 처리 
x_data_test = pd.concat([test_onehot,x_data], axis=1)
x_data_test =scaler.transform(x_data_test)
sample = pd.read_csv("sample_submission.csv")
sample['SalePrice']=sess.run(hypothesis, feed_dict={X:x_data_test})

In [ ]:
log_rmse(hy_val.reshape(-1,), y_train)
# 결과 : 0.155009267656005